In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
# Set the seed for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Path to the dataset directory
dataset_path = '/content/drive/MyDrive/Watermark Detection/'

# Constants
img_width = 128
img_height = 128
batch_size = 32
num_epochs = 10


In [4]:
# Load and preprocess the dataset
def load_dataset():
    images = []
    labels = []

    watermark_images_path = os.path.join(dataset_path, 'watermark/')
    non_watermark_images_path = os.path.join(dataset_path, 'no-watermark/')

    # Load watermark images
    for image_name in os.listdir(watermark_images_path):
        image_path = os.path.join(watermark_images_path, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (img_width, img_height))
        images.append(image)
        labels.append(1)  # Watermark class label
    # Load non-watermark images
    for image_name in os.listdir(non_watermark_images_path):
        image_path = os.path.join(non_watermark_images_path, image_name)
        image = cv2.imread(image_path)
        image = cv2.resize(image, (img_width, img_height))
        images.append(image)
        labels.append(0)  # Non-watermark class label
    # Convert lists to arrays
    images = np.array(images, dtype=np.float32)
    labels = np.array(labels, dtype=np.int32)
    return images, labels

In [5]:
# Split the dataset into training and testing sets
def split_dataset(images, labels):
    x_train, x_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)
    return x_train, x_test, y_train, y_test

In [6]:
# Load the dataset
images, labels = load_dataset()

# Split the dataset
x_train, x_test, y_train, y_test = split_dataset(images, labels)

# Normalize pixel values between 0 and 1
x_train = x_train / 255.0
x_test = x_test / 255.0


In [7]:
# Define the model architecture
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout


model = tf.keras.Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(img_width, img_height, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [8]:
# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [9]:
# Train the model
history = model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=num_epochs, batch_size=batch_size)

Epoch 1/10
165/165 [==============================] - 220s 1s/step - loss: 0.6956 - accuracy: 0.5008 - val_loss: 0.6940 - val_accuracy: 0.4772
Epoch 2/10
165/165 [==============================] - 208s 1s/step - loss: 0.6933 - accuracy: 0.5072 - val_loss: 0.6932 - val_accuracy: 0.4909
Epoch 3/10
165/165 [==============================] - 211s 1s/step - loss: 0.6924 - accuracy: 0.5123 - val_loss: 0.6938 - val_accuracy: 0.4772
Epoch 4/10
165/165 [==============================] - 211s 1s/step - loss: 0.6933 - accuracy: 0.4981 - val_loss: 0.6936 - val_accuracy: 0.4772
Epoch 5/10
165/165 [==============================] - 210s 1s/step - loss: 0.6933 - accuracy: 0.4983 - val_loss: 0.6930 - val_accuracy: 0.5228
Epoch 6/10
165/165 [==============================] - 212s 1s/step - loss: 0.6935 - accuracy: 0.4983 - val_loss: 0.6939 - val_accuracy: 0.4772
Epoch 7/10
165/165 [==============================] - 213s 1s/step - loss: 0.6938 - accuracy: 0.4973 - val_loss: 0.6940 - val_accuracy: 0.4772

In [11]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(x_test, y_test)
print(f'Test loss: {loss:.4f}')
print(f'Test accuracy: {accuracy:.4f}')

42/42 [==============================] - 16s 373ms/step - loss: 0.4825 - accuracy: 0.7701
Test loss: 0.4825
Test accuracy: 0.7701


In [ ]:
training_accuracy = history.history['accuracy']
avg_training_accuracy = sum(training_accuracy) / len(training_accuracy)
print(f'Average training accuracy: {avg_training_accuracy:.4f}')

Average training accuracy: 0.9428


In [ ]:
model.save('model.h5')